In [1]:
#from bs4 import BeautifulSoup
#import requests
import pandas as pd
import unidecode
from pathlib import Path
import numpy as np

# Doações

In [2]:
cand_doacoes = pd.read_csv("doacoes/receitas_candidatos_2014_brasil.txt",sep=';', header=0, encoding = 'latin_1', converters={'CPF do candidato': lambda x: str(x), 'CPF/CNPJ do doador': lambda x: str(x)}, decimal=',')

In [3]:
# Apaga colunas que não serão usadas
del cand_doacoes['Cód. Eleição']
del cand_doacoes['Desc. Eleição']
del cand_doacoes['CNPJ Prestador Conta']
del cand_doacoes['Data e hora']
del cand_doacoes['Sequencial Candidato']
del cand_doacoes['Numero candidato']
del cand_doacoes['Numero Recibo Eleitoral']
del cand_doacoes['Numero do documento']
del cand_doacoes['Nome do doador (Receita Federal)']
del cand_doacoes['Sigla UE doador']
del cand_doacoes['Número partido doador']
del cand_doacoes['Número candidato doador']
del cand_doacoes['Cod setor econômico do doador']
del cand_doacoes['Setor econômico do doador']
del cand_doacoes['Data da receita']
del cand_doacoes['Setor econômico do doador originário']
del cand_doacoes['Nome do doador originário (Receita Federal)']

In [4]:
cand_doacoes_d_2014 = cand_doacoes[(cand_doacoes['Cargo'] == 'Deputado Estadual') & (cand_doacoes['UF'] == 'SP')]

In [5]:
#cand_doacoes_d_2014 = cand_doacoes_d_2014[(cand_doacoes_d_2014['CPF/CNPJ do doador'] != 0.0)]

In [6]:
#cand_doacoes_d_2014 = cand_doacoes_d_2014[pd.notnull(cand_doacoes_d_2014['CPF/CNPJ do doador'])]

In [7]:
cand_doacoes_d_2014.columns = ['uf', 'partido_eleicao', 'cargo', 'nome_completo', 'cpf', 'cpf_cnpj_doador', 'nome_doador', 'valor', 'tipo_receita', 'fonte_recurso', 'especie_recurso', 'descricao', 'cpf_cnpj_doador_originario', 'nome_doador_originario', 'tipo_doador_originario']

In [8]:
def limpeza1(linha):
    documento = linha["cpf_cnpj_doador"]
    tipo = linha["tipo_receita"]
    if len(documento) > 11 and tipo != "Recursos de outros candidatos/comitês":
        return documento[:8]
    elif tipo == "Recursos de outros candidatos/comitês":
        return "Recursos de outros candidatos/comitês"
    else:
        return "documento_cpf"

def limpeza2(linha):
    documento = linha["cpf_cnpj_doador_originario"]
    if len(documento) > 11 and documento != "#NULO" and documento != None:
        return documento[:8]
    elif documento == "#NULO" or documento == None:
        return "#NULO"
    else:
        return "documento_cpf"

In [9]:
cand_doacoes_d_2014['cnpj_doador_raiz'] = cand_doacoes_d_2014.apply(limpeza1, axis = 1)
cand_doacoes_d_2014['cnpj_doador_originario_raiz'] = cand_doacoes_d_2014.apply(limpeza2, axis = 1)

/home/reinaldo/Documentos/Code/volt/alesp/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/reinaldo/Documentos/Code/volt/alesp/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
cand_doacoes_d_2014.rename(columns={'valor':'valor_doado'}, inplace=True)

/home/reinaldo/Documentos/Code/volt/alesp/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [11]:
cand_doacoes_d_2014.reset_index().head(5)

,index,uf,partido_eleicao,cargo,nome_completo,cpf,cpf_cnpj_doador,nome_doador,valor_doado,tipo_receita,fonte_recurso,especie_recurso,descricao,cpf_cnpj_doador_originario,nome_doador_originario,tipo_doador_originario,cnpj_doador_raiz,cnpj_doador_originario_raiz
0,23,SP,PT,Deputado Estadual,ALENCAR SANTANA BRAGA,05544839808,58809203000103,BRASILIAN COLOR I T,2500.0,Recursos de pessoas jurídicas,Nao especificado,Transferência eletrônica,#NULO,#NULO,#NULO,#NULO,58809203,#NULO
1,51,SP,PSDB,Deputado Estadual,JOAO CARLOS CARAMEZ,41352009820,43483247000119,HIDROCONSULT CONSULTORIA ESTUDOS E PROJETOS LTDA,3000.0,Recursos de pessoas jurídicas,Nao especificado,Cheque,#NULO,#NULO,#NULO,#NULO,43483247,#NULO
2,114,SP,PT do B,Deputado Estadual,PAULO ROBERTO ROSENO JUNIOR,05488774831,20557696000169,JORDÃO MOISES FABREGA,447.5,Recursos de outros candidatos/comitês,Outros Recursos nao descritos,Estimado,REFERENTE A DOBRADA DE 15 CAVALETES E 1 PLACA ...,00639041000122,INMETRA MEDICINA E SEGURANÇA DO TRABALHO LTDA,J,Recursos de outros candidatos/comitês,00639041
3,220,SP,PV,Deputado Estadual,BRUNO AREVALO GANEM,35205827831,07334221000145,WBB CONSTRUÇÕES LTDA,840.0,Recursos de pessoas jurídicas,Nao especificado,Estimado,CESSÃO DO VEÍCULO VW KOMBI 2010 - PLACAS DSS-5115,#NULO,#NULO,#NULO,07334221,#NULO
4,221,SP,PSC,Deputado Estadual,MARIA HELENA JURADO MELILLO,22678150879,20559333000162,ELEIÇÃO 2014 ROBERTO MIGUEL REY JUNIOR DEPUTAD...,220.0,Recursos de outros candidatos/comitês,Outros Recursos nao descritos,Estimado,40 POR CENTO DE 50 CONVITES 14X21 CM PAPEL 300...,#NULO,#NULO,#NULO,Recursos de outros candidatos/comitês,#NULO


# Candidatos

In [12]:
def _reader_2014(fname):
    return pd.read_csv(fname, sep=';', header=None, encoding='latin_1', converters={13: lambda x: str(x), 11: lambda x: str(x)})

In [13]:
folder = Path("candidatos_2014")
candidatos_2014 = pd.concat([
    _reader_2014(txt)
    for txt in folder.glob("*.txt")
])

In [14]:
# Apaga as colunas que não serão usadas
del candidatos_2014[0]
del candidatos_2014[1]
del candidatos_2014[2]
del candidatos_2014[3]
del candidatos_2014[4]
del candidatos_2014[6]
del candidatos_2014[7]
del candidatos_2014[8]
del candidatos_2014[12]
del candidatos_2014[15]
del candidatos_2014[16]
del candidatos_2014[17]
del candidatos_2014[19]
del candidatos_2014[20]
del candidatos_2014[21]
del candidatos_2014[22]
del candidatos_2014[23]
del candidatos_2014[24]
del candidatos_2014[25]
del candidatos_2014[26]
del candidatos_2014[27]
del candidatos_2014[28]
del candidatos_2014[29]
del candidatos_2014[30]
del candidatos_2014[31]
del candidatos_2014[32]
del candidatos_2014[33]
del candidatos_2014[34]
del candidatos_2014[35]
del candidatos_2014[36]
del candidatos_2014[37]
del candidatos_2014[38]
del candidatos_2014[39]
del candidatos_2014[40]
del candidatos_2014[41]
del candidatos_2014[42]
del candidatos_2014[43]
del candidatos_2014[45]

In [15]:
candidatos_2014.columns = ['uf', 'cargo', 'nome_completo', 'sequencial', 'cpf', 'nome_urna', 'partido_eleicao', 'situacao']

In [16]:
eleitos = candidatos_2014[(candidatos_2014['situacao'] == 'ELEITO POR QP') | (candidatos_2014['situacao'] == 'ELEITO POR MÉDIA') | (candidatos_2014['situacao'] == 'ELEITO')]

In [17]:
eleitos_d_2014 = eleitos[(eleitos['cargo'] == 'DEPUTADO ESTADUAL') & (eleitos['uf'] == 'SP')]

In [18]:
eleitos_d_2014.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94 entries, 25 to 3650
Data columns (total 8 columns):
uf                 94 non-null object
cargo              94 non-null object
nome_completo      94 non-null object
sequencial         94 non-null object
cpf                94 non-null object
nome_urna          94 non-null object
partido_eleicao    94 non-null object
situacao           94 non-null object
dtypes: object(8)
memory usage: 6.6+ KB


In [19]:
eleitos_d_2014.to_csv('resultados/deputados_eleitos_2014_sp.csv', index=False) 

In [20]:
eleitos_d_doadores = pd.merge(cand_doacoes_d_2014, eleitos_d_2014, left_on='cpf', right_on='cpf')

In [21]:
eleitos_d_doadores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12070 entries, 0 to 12069
Data columns (total 24 columns):
uf_x                           12070 non-null object
partido_eleicao_x              12070 non-null object
cargo_x                        12070 non-null object
nome_completo_x                12070 non-null object
cpf                            12070 non-null object
cpf_cnpj_doador                12070 non-null object
nome_doador                    12070 non-null object
valor_doado                    12070 non-null float64
tipo_receita                   12070 non-null object
fonte_recurso                  12070 non-null object
especie_recurso                12070 non-null object
descricao                      12070 non-null object
cpf_cnpj_doador_originario     12070 non-null object
nome_doador_originario         12070 non-null object
tipo_doador_originario         12070 non-null object
cnpj_doador_raiz               12070 non-null object
cnpj_doador_originario_raiz    12070 non-n

In [22]:
eleitos_d_doadores.to_csv('resultados/doacoes_deputados_alesp_2014.csv', index=False, decimal = ',')

# Cruza com despesas de deputados

In [23]:
eleitos_d_doadores = pd.read_csv("resultados/doacoes_deputados_alesp_2014.csv", sep=',',encoding = 'utf-8', converters={'cpf_cnpj_doador': lambda x: str(x), 'cpf': lambda x: str(x),  'cpf_cnpj_doador_originario': lambda x: str(x), 'cnpj_doador_raiz': lambda x: str(x), 'cnpj_doador_originario_raiz': lambda x: str(x), 'sequencial': lambda x: str(x)}, decimal=',')

In [24]:
eleitos_d_doadores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12070 entries, 0 to 12069
Data columns (total 24 columns):
uf_x                           12070 non-null object
partido_eleicao_x              12070 non-null object
cargo_x                        12070 non-null object
nome_completo_x                12070 non-null object
cpf                            12070 non-null object
cpf_cnpj_doador                12070 non-null object
nome_doador                    12070 non-null object
valor_doado                    12070 non-null float64
tipo_receita                   12070 non-null object
fonte_recurso                  12070 non-null object
especie_recurso                12070 non-null object
descricao                      12070 non-null object
cpf_cnpj_doador_originario     12070 non-null object
nome_doador_originario         12070 non-null object
tipo_doador_originario         12070 non-null object
cnpj_doador_raiz               12070 non-null object
cnpj_doador_originario_raiz    12070 non-n

In [25]:
# Total valor doado para os 94 eleitos

In [26]:
eleitos_d_doadores['valor_doado'].sum()

101889797.16

In [ ]:
# Busca de doador pelo CNPJ raiz

In [16]:
nome = eleitos_d_doadores[(eleitos_d_doadores['cnpj_doador_raiz'] == '43283811')]
nome

,uf_x,partido_eleicao_x,cargo_x,nome_completo_x,cpf,cpf_cnpj_doador,nome_doador,valor_doado,tipo_receita,fonte_recurso,...,tipo_doador_originario,cnpj_doador_raiz,cnpj_doador_originario_raiz,uf_y,cargo_y,nome_completo_y,sequencial,nome_urna,partido_eleicao_y,situacao
1925,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,2635.0,Recursos de pessoas jurídicas,Nao especificado,...,#NULO,43283811,#NULO,SP,DEPUTADO ESTADUAL,FERNANDO CAPEZ,250000001510,FERNANDO CAPEZ,PSDB,ELEITO POR QP
5153,SP,PC do B,Deputado Estadual,LECI BRANDÃO DA SILVA,18264590730,43283811000150,KALUNGA COMÉRCIO E INDÚSTRIA GRÁFICA LTDA,2635.0,Recursos de pessoas jurídicas,Nao especificado,...,#NULO,43283811,#NULO,SP,DEPUTADO ESTADUAL,LECI BRANDÃO DA SILVA,250000000071,LECI BRANDÃO,PC do B,ELEITO POR QP
9209,SP,PV,Deputado Estadual,EDSON DE OLIVEIRA GIRIBONI,98361325891,43283811000150,KALUNGA COMÉRCIO E INDÚSTRIA GRÁFICA LTDA,5270.0,Recursos de pessoas jurídicas,Nao especificado,...,#NULO,43283811,#NULO,SP,DEPUTADO ESTADUAL,EDSON DE OLIVEIRA GIRIBONI,250000001019,GIRIBONI,PV,ELEITO POR QP


In [27]:
despesas = pd.read_csv("resultados/despesas_dep_est_sp_8_junho.csv", sep=',',encoding = 'utf-8', converters={'CNPJ': lambda x: str(x), 'cnpj_raiz_fornecedor': lambda x: str(x)}, decimal=',')

In [28]:
despesas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453000 entries, 0 to 452999
Data columns (total 9 columns):
Ano                     453000 non-null int64
CNPJ                    453000 non-null object
Deputado                453000 non-null object
Fornecedor              453000 non-null object
Matricula               453000 non-null int64
Mes                     453000 non-null int64
Tipo                    453000 non-null object
Valor                   453000 non-null float64
cnpj_raiz_fornecedor    453000 non-null object
dtypes: float64(1), int64(3), object(5)
memory usage: 31.1+ MB


In [29]:
despesas.reset_index()

,index,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor
0,0,2015,71806251000106,ABELARDO CAMARINHA,AUTO POSTO DE SERVIÇOS MARV LTDA,300257,3,A - COMBUSTÍVEIS E LUBRIFICANTES,200.00,71806251
1,1,2015,68064740000125,ABELARDO CAMARINHA,PAPER FACE COMERCIAL LTDA,300257,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,295.40,68064740
2,2,2015,22145388877,ABELARDO CAMARINHA,LARA SERVINO VARGAS ABREU,300257,3,N - MORADIA,2850.00,documento_cpf_
3,3,2015,10915883000113,ABELARDO CAMARINHA,CENTRO AUTOMOTIVO TOP 23 LTDA,300257,4,A - COMBUSTÍVEIS E LUBRIFICANTES,100.00,10915883
4,4,2015,71806251000106,ABELARDO CAMARINHA,AUTO POSTO DE SERVIÇOS MARV LTDA,300257,4,A - COMBUSTÍVEIS E LUBRIFICANTES,195.01,71806251
5,5,2015,06222505000187,ABELARDO CAMARINHA,WAGNER HUBERT EPP,300257,4,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,200.00,06222505
6,6,2015,10455524000120,ABELARDO CAMARINHA,ROGÉRIO GODOY CARIMBOS - ME,300257,4,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,60.00,10455524
7,7,2015,04206050000180,ABELARDO CAMARINHA,TIM CELULAR SA,300257,4,"H - SERV.UTIL.PÚBLICA (TELEF.MÓVEL/FIXA, ENERG...",719.02,04206050
8,8,2015,14691478000110,ABELARDO CAMARINHA,AGENCIA DE SERV POSTAIS AVARÉ LTDA,300257,4,J - SERVIÇOS DE COMUNICAÇÃO,719.94,14691478
9,9,2015,22145388877,ABELARDO CAMARINHA,LARA SERVINO VARGAS ABREU,300257,4,N - MORADIA,2850.00,documento_cpf_


In [30]:
despesas.rename(columns={'Valor':'valor_fornecedor'}, inplace=True)

In [31]:
despesas_atuais = despesas[(despesas['Ano'] == 2015) | (despesas['Ano'] == 2016) | (despesas['Ano'] == 2017) | (despesas['Ano'] == 2018)]

In [32]:
def limpeza3(linha):
    documento = linha["CNPJ"]
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf_"

In [33]:
despesas_atuais['cnpj_raiz_fornecedor'] = despesas_atuais.apply(limpeza3, axis = 1)

/home/reinaldo/Documentos/Code/volt/alesp/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
# Total de despesas

In [35]:
despesas_atuais['valor_fornecedor'].sum()

75154853.75000001

In [51]:
despesas_atuais.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75868 entries, 0 to 445457
Data columns (total 9 columns):
Ano                     75868 non-null int64
CNPJ                    75868 non-null object
Deputado                75868 non-null object
Fornecedor              75868 non-null object
Matricula               75868 non-null int64
Mes                     75868 non-null int64
Tipo                    75868 non-null object
valor_fornecedor        75868 non-null float64
cnpj_raiz_fornecedor    75868 non-null object
dtypes: float64(1), int64(3), object(5)
memory usage: 5.8+ MB


In [37]:
# Despesas só com CNPJ

In [38]:
seleciona_cnpjs = despesas_atuais[(despesas_atuais['cnpj_raiz_fornecedor'] != 'documento_cpf_')]
seleciona_cnpjs['valor_fornecedor'].sum()

61514114.17

# Doações

In [53]:
# Cruza para pegar todas doações diretas e não repetir fornecedores

In [39]:
despesas_doadores_diretos = pd.merge(eleitos_d_doadores, despesas_atuais.drop_duplicates('cnpj_raiz_fornecedor'), left_on='cnpj_doador_raiz', right_on='cnpj_raiz_fornecedor')

In [40]:
despesas_doadores_diretos.reset_index().head()

,index,uf_x,partido_eleicao_x,cargo_x,nome_completo_x,cpf,cpf_cnpj_doador,nome_doador,valor_doado,tipo_receita,...,situacao,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,valor_fornecedor,cnpj_raiz_fornecedor
0,0,SP,PT,Deputado Estadual,ALENCAR SANTANA BRAGA,05544839808,60863032000142,SAVOY IMOBILIARIA CONST LTDA,5000.0,Recursos de pessoas jurídicas,...,ELEITO POR QP,2015,60863032000142,ADRIANO DIOGO,SAVOY IMOBILIARIA CONSTRUTORA LTDA.,300420,1,L - LOCAÇÃO DE BENS IMÓVEIS,4634.63,60863032
1,1,SP,PT,Deputado Estadual,ALENCAR SANTANA BRAGA,05544839808,46319901000141,JOIA DE GUARULHOS POSTO DE SERVIÇOS LTDA,1000.0,Recursos de pessoas jurídicas,...,ELEITO POR QP,2015,46319901000141,CARLOS NEDER,JOIA DE GUARULHOS DE SERVICOS LTDA,300446,2,A - COMBUSTÍVEIS E LUBRIFICANTES,120.86,46319901
2,2,SP,PT,Deputado Estadual,ALENCAR SANTANA BRAGA,05544839808,45827425000107,SUPERMERCADOS IRMÃOS LOPES S/A,10000.0,Recursos de pessoas jurídicas,...,ELEITO POR QP,2015,45827425000530,ANALICE FERNANDES,SUPERMERCADOS IRMAOS LOPES S/A,300431,9,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,139.49,45827425
3,3,SP,PT,Deputado Estadual,ALENCAR SANTANA BRAGA,05544839808,60437647000107,SILVA E BARBOSA COM ALIM LTDA,5000.0,Recursos de pessoas jurídicas,...,ELEITO POR QP,2017,60437647001936,GIL LANCASTER,SILVA E BARBOSA COMERCIO DE ALIMENTOS LTDA,300585,4,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,193.47,60437647
4,4,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,60437647000107,SILVA E BARBOSA COMERCIO DE ALIMENTOS LTDA,5000.0,Recursos de pessoas jurídicas,...,ELEITO POR QP,2017,60437647001936,GIL LANCASTER,SILVA E BARBOSA COMERCIO DE ALIMENTOS LTDA,300585,4,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,193.47,60437647


In [42]:
# Total de doações dos doadores que são fornecedores

In [43]:
despesas_doadores_diretos['valor_doado'].sum()

904691.98

In [44]:
# Quem mais doou entre os fornecedores

In [45]:
despesas_doadores_diretos.groupby("nome_doador").valor_doado.sum().reset_index().sort_values("valor_doado", ascending=False)

,nome_doador,valor_doado
54,UNIQUE - SERVIÇOS DE HOTELARIA E ALIM.· COM. E...,100000.00
53,TENDA ATACADO LTDA.,100000.00
8,AUTO POSTO POLISERVIÇOS I LTDA,100000.00
39,PORTO SEGURO COMPANHIA DE SEGUROS GERAIS,75000.00
36,ORGANIZACAO COMERCIAL LAGOA AZUL LTDA,60000.00
15,COOP - COOPERATIVA DE CONSUMO,60000.00
17,COOPERATIVA DOS PLANTADORES DE CANA DO OESTE D...,50000.00
16,COOP. PLANTADORES DE CANA DO OESTE DO EST DE S...,50000.00
12,CIA BRASILEIRA DE DISTRIBUIÇÃO,50000.00
44,ROLDÃO AUTO SERVIÇO COMÉRCIO DE ALIMENTOS LTDA,40000.00


In [ ]:
# Tabela dinâmica com ano, deputado e fornecedores - somando doacoes - (não precisaria de ano)

In [68]:
pd.pivot_table(despesas_doadores_diretos,index=["Ano","Deputado", "Fornecedor"], values=["valor_doado"], aggfunc=np.sum)

valor_doado
Ano  Deputado              Fornecedor                                                   
2015 ADILSON ROSSI         KALUNGA COM INDUSTRIA GRAFICA LTDA                   10540.00
     ADRIANO DIOGO         SAVOY IMOBILIARIA CONSTRUTORA LTDA.                   5000.00
     AFONSO LOBATO         GRAFICA ELITE                                           70.00
                           TENDA ATACADO                                       100000.00
     ANALICE FERNANDES     SUPERMERCADOS IRMAOS LOPES S/A                       10000.00
     ANGELO PERUGINI       GOOD BOM SUPERMERCADOS LTDA                           2864.25
     BETH SAHÃO            REDE SEG GRAFICA E EDITORA EIRELI EPP                  100.00
     CARLOS NEDER          JOIA DE GUARULHOS DE SERVICOS LTDA                    1000.00
     CAUÊ MACRIS           AUTO POSTO B.B.MOGI LTDA                              2324.25
     CELINO CARDOSO        AUTO POSTO LAERCIO DE MAIRIPORÃ LTDA                  3551.31
     CELSO GIGLIO          GRÁFICA TRES PONTAS LTDA - EPP                         860.00
     CHICO SARDELLI        CRONY ARTES GRAFICAS LTDA ME                           460.00
                           JOSE H. G. DE OLIVEIRA EDITORA ME                     1260.00
     CLÉLIA GOMES          COOP. PLANT. DE CANA OESTE EST. SAO PAULO           100000.00
     ED THOMAS             HANNA INCORPORAÇÕES E VENDAS LTDA                    15000.00
     ESTEVAM GALVÃO        INTEGRA EDITORA GRAFICA LTDA                           450.00
                           PORTO SEGURO CIA DE SEGUROS GERAIS                   75000.00
     GERALDO CRUZ          PASARGADA SERVIÇOS AUTOMOTIVOS LTDA                   3000.00
     GILSON DE SOUZA       ALGAR TELECOM S/A                                      160.00
     IGOR SOARES           ALPHAMARK PROPAGANDA E MARKETING LTDA                 2000.00
                           AUTO POSTO POLISERVIÇOS I LTDA                      100000.00
     ITAMAR BORGES         FERNANDO LUIZ LAUDINO & CIA LTDA - ME                 2700.00
     LUIZ CARLOS GONDIM    GRANJA VIANA AUTO POSTO EIRELI                        1000.00
     MARIA LÚCIA AMARY     COM DERIV PETROLEO RECREIO CAMPESTRE LTDA              893.70
     ORLANDO MORANDO       EXPRESSÃO SANTO ANDRE GRAFICA E EDITORA LTDA EPP       325.00
2016 ADILSON ROSSI         CIA BRASILEIRA DE DISTRIBUIÇÃO                       50000.00
     ANALICE FERNANDES     ORGANIZAÇAO COML. LAGO AZUL LTDA                     60000.00
                           SUPERMERCADOS KACULA LTDA                            10000.00
     ANGELO PERUGINI       ASPIL INFORMATICA LTDA EPP                           10400.00
                           COMERCIO DE COMBUSTIVEIS PARQUE GABRIEL LTDA         25000.00
     ATILA JACOMUSSI       COOP COOPERATIVA DE CONSUMO                          60000.00
     BARROS MUNHOZ         SUPERMERCADOS CAVICCHIOLLI LTDA.                      2994.27
     CARLOS NEDER          AUTO POSTO PAGUE MENOS LTDA                           8493.20
     CAUÊ MACRIS           POSTO JENNER LTDA                                     1000.00
     CLÉLIA GOMES          SAVEGNAGO SUPERMERCADOS LTDA                         25000.00
     CORONEL TELHADA       CENTRO AUTOMOTIVO ELOS DE OURO LTDA                   1000.00
     FERNANDO CURY         HABIBS BOTUCATU                                       1500.00
                           PROVIDE COMERCIO DE VEICULOS BOTUCATU LTDA            4200.00
     LUIZ CARLOS GONDIM    DIARIO DO ALTO TIETE EMPRESA JORNALISTICA             2448.00
                           MOGI NEWS EMPRESA JORNALÍSTICA                        2448.00
     MARCOS DAMASIO        UNIQUE SERV.DE HOTELARIA ALIM.COM PART.S/A          100000.00
     MARCOS NEVES          REIS OFFICE PRODUCTS COMERCIAL LTDA                  15000.00
     MÁRCIO CAMARGO        SUPERMERCADO PEDROSAO LTDA                            1000.00
2017 ALENCAR SANTANA BRAGA ATACADISTA ROLDÃO                            

In [46]:
despesas_doadores_diretos.to_csv('resultados/doacoes_deputados_sp_fornecedores.csv', index=False, decimal = ',')

# PF

In [47]:
despesas_atuaisPF = despesas_atuais[(despesas_atuais['cnpj_raiz_fornecedor'] == 'documento_cpf_')]

In [48]:
despesas_doadores_diretosPF = pd.merge(eleitos_d_doadores, despesas_atuaisPF.drop_duplicates('CNPJ'), left_on='cpf_cnpj_doador', right_on='CNPJ')

In [50]:
# Total de doações de fornecedores PF

In [51]:
despesas_doadores_diretosPF['valor_doado'].sum()

407744.0

In [53]:
# Maiores doadores PF que são fornecedores

In [54]:
despesas_doadores_diretosPF.groupby("nome_doador").valor_doado.sum().reset_index().sort_values("valor_doado", ascending=False)

,nome_doador,valor_doado
6,ROBERTO MASSAFERA,315520.0
4,IGOR SOARES EBERT,63000.0
0,ANTONIO CABEZA SASTRE,10500.0
2,CLOVIS SILES GALVÃO,10500.0
5,MARIA ELIZA INTERDONATO NUNES VIANA,3000.0
3,EUVALDO RODRIGUEZ ALBALADEJO,2500.0
7,ROBERTO TAVARES DE CARVALHO,2000.0
1,CLEUSA ANANIAS DE ARAUJO,724.0


In [ ]:
# ROBERTO MASSAFERA é um deputado, fornecedor e doou para si mesmo

In [77]:
nome = despesas_doadores_diretosPF[(despesas_doadores_diretosPF['nome_doador'] == 'ROBERTO MASSAFERA')]
nome

,uf_x,partido_eleicao_x,cargo_x,nome_completo_x,cpf,cpf_cnpj_doador,nome_doador,valor_doado,tipo_receita,fonte_recurso,...,situacao,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,valor_fornecedor,cnpj_raiz_fornecedor
6,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,100000.0,Recursos próprios,Nao especificado,...,ELEITO POR QP,2017,02674980800,ROBERTO MASSAFERA,CIRO BERBES JUNIOR,300478,8,N - MORADIA,2500.0,documento_cpf_
7,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,20000.0,Recursos próprios,Nao especificado,...,ELEITO POR QP,2017,02674980800,ROBERTO MASSAFERA,CIRO BERBES JUNIOR,300478,8,N - MORADIA,2500.0,documento_cpf_
8,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,30000.0,Recursos próprios,Nao especificado,...,ELEITO POR QP,2017,02674980800,ROBERTO MASSAFERA,CIRO BERBES JUNIOR,300478,8,N - MORADIA,2500.0,documento_cpf_
9,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,2000.0,Recursos próprios,Nao especificado,...,ELEITO POR QP,2017,02674980800,ROBERTO MASSAFERA,CIRO BERBES JUNIOR,300478,8,N - MORADIA,2500.0,documento_cpf_
10,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,640.0,Recursos próprios,Nao especificado,...,ELEITO POR QP,2017,02674980800,ROBERTO MASSAFERA,CIRO BERBES JUNIOR,300478,8,N - MORADIA,2500.0,documento_cpf_
11,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,2880.0,Recursos próprios,Nao especificado,...,ELEITO POR QP,2017,02674980800,ROBERTO MASSAFERA,CIRO BERBES JUNIOR,300478,8,N - MORADIA,2500.0,documento_cpf_
12,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,10000.0,Recursos próprios,Nao especificado,...,ELEITO POR QP,2017,02674980800,ROBERTO MASSAFERA,CIRO BERBES JUNIOR,300478,8,N - MORADIA,2500.0,documento_cpf_
13,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,30000.0,Recursos próprios,Nao especificado,...,ELEITO POR QP,2017,02674980800,ROBERTO MASSAFERA,CIRO BERBES JUNIOR,300478,8,N - MORADIA,2500.0,documento_cpf_
14,SP,PSDB,Deputado Estadual,ROBERTO MASSAFERA,02674980800,02674980800,ROBERTO MASSAFERA,120000.0,Recursos próprios,Nao especificado,...,ELEITO POR QP,2017,02674980800,ROBERTO MASSAFERA,CIRO BERBES JUNIOR,300478,8,N - MORADIA,2500.0,documento_cpf_


# Despesas

In [70]:
# Pega todos os valores de fornecedores que são doadores

In [55]:
despesas_doadores_diretos_f = pd.merge(eleitos_d_doadores.drop_duplicates('cnpj_doador_raiz'), despesas_atuais, left_on='cnpj_doador_raiz', right_on='cnpj_raiz_fornecedor')

In [ ]:
# Soma de despesas pagas aos fornecedores/doadores 

In [56]:
despesas_doadores_diretos_f['valor_fornecedor'].sum()

1816310.35

In [57]:
# Tabela dinâmica por ano, deputado, cnpj raiz e fornecedor

In [58]:
pd.pivot_table(despesas_doadores_diretos_f,index=["Ano","Deputado", "cnpj_raiz_fornecedor", "Fornecedor"], values=["valor_fornecedor"], aggfunc=np.sum)

valor_fornecedor
Ano  Deputado              cnpj_raiz_fornecedor Fornecedor                                                 
2015 ADILSON ROSSI         43283811             KALUNGA COM INDUSTRIA GRAFICA LTDA                   922.59
     ADRIANO DIOGO         60863032             SAVOY IMOBILIARIA CONSTRUTORA LTDA.                12427.47
     AFONSO LOBATO         01157555             TENDA ATACADO                                        170.50
                           47508411             PÃO DE AÇÚCAR                                         10.44
                           56035009             GRAFICA ELITE                                      29200.00
     ALDO DEMARCHI         43283811             KALUNGA COMÉRCIO E INDUSTRIA GRÁFICA LTDA            493.70
                                                KALUNGA COMÉRCIOINDUSTRIA GRÁFICA LTDA              1334.78
     ALENCAR SANTANA BRAGA 43283811             KALUNGA LTDA.                                        264.90
     ALEXANDRE DA FARMÁCIA 43283811             KALUNGA COMERCIO E IND GRAFICA LTDA                   27.80
                                                KALUNGA COMERCIO E IND. GRAFICA LTDA.                697.70
     ALEXANDRE PEREIRA     43283811             KALUNGA  COM E IND. GRAF. LTDA -SP                   585.63
                                                KALUNGA COM  IND. GRAFICA LTDA                       327.90
                                                KALUNGA COM IND GRAFICA LTDA                          95.70
                                                KALUNGA COMERCIO E IND GRAFICA LTDA                   39.00
                                                KALUNGA TATUAPÉ                                       37.80
                                                KALUNGA VILA GUILHERME                               199.70
     ANA DO CARMO          43283811             KALUNGA                                              805.99
     ANALICE FERNANDES     43283811             KALUNGA COMERCIO E IND. GRAFICA LTDA                 159.90
                           45827425             SUPERMERCADOS IRMAOS LOPES S/A                       139.49
                           47508411             CIA BRASILEIRA DE DISTRIBUIÇÃO                       266.18
     ANDRÉ DO PRADO        43283811             KALUNGA COMÉRCIO INSDÚSTRIA GRÁFICA LTDA            3728.24
                           47508411             CIA BRASILEIRA DE DISTRIBUIÇÃO                        82.51
     ANDRÉ SOARES          43283811             KALUNGA  COMERCIO E IND GRAFICA LTDA                 184.19
                                                KALUNGA COMÉRCIO E INDÚSTRIA GRÁFICA LTDA            555.55
                           47508411             CIA BRASILEIRA DE DISTRIBUIÇÃO                      1577.05
                           72911837             ORGANIZAÇÃO COML LAGO AZUL LTDA                      132.00
     ANGELO PERUGINI       43283811             KALUNGA COMÉRCIO E IND. GRAFICA LTDA                 830.16
                           56794852             GOOD BOM SUPERMERCADOS LTDA                           44.93
     ANTONIO SALIM CURIATI 43283811             KALUNGA COMERCIO E IND. GRAFICA LTDA                2131.32
                                                KALUNGA COMERCIO E IND. GRAFICA LTDA.                539.97
...                                                                                                     ...
2018 PEDRO TOBIAS          43283811             KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA            201.60
                                                KALUNGA COMERCIO INDUSTRIA GRAFICA LTDA              277.40
     RAMALHO DA CONSTRUÇÃO 61699179             POSTO JENNER LTDA                                    659.00
     RAUL MARCELO          43283811             KALUNGA COM. E IND. GRÁFICA LTDA                     164.60
                                                KALUNGA COMÉRCIO E INDÚSTRIA GRÁFICA LTDA      

In [ ]:
# Soma de despesas pelo CNPJ raiz dos fornecedores

In [60]:
despesas_doadores_diretos_f.groupby("cnpj_raiz_fornecedor").valor_fornecedor.sum().reset_index().sort_values("valor_fornecedor", ascending=False)

,cnpj_raiz_fornecedor,valor_fornecedor
24,43283811,448989.69
21,15045533,224415.00
34,54450705,215436.40
31,47828827,215196.00
7,04867562,125978.40
32,52837408,112150.00
10,05790610,88630.58
37,56035009,67080.00
0,00158540,57750.00
17,10928875,47604.71


In [61]:
nome = despesas_doadores_diretos_f[(despesas_doadores_diretos_f['cnpj_raiz_fornecedor'] == '43283811')]
nome

,uf_x,partido_eleicao_x,cargo_x,nome_completo_x,cpf,cpf_cnpj_doador,nome_doador,valor_doado,tipo_receita,fonte_recurso,...,situacao,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,valor_fornecedor,cnpj_raiz_fornecedor
313,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,2635.0,Recursos de pessoas jurídicas,Nao especificado,...,ELEITO POR QP,2015,43283811001806,ADILSON ROSSI,KALUNGA COM INDUSTRIA GRAFICA LTDA,300432,12,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,922.59,43283811
314,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,2635.0,Recursos de pessoas jurídicas,Nao especificado,...,ELEITO POR QP,2017,43283811002284,ADILSON ROSSI,KALUNGA COM E IND. GRAFICA LTDA,300432,5,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,52.10,43283811
315,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,2635.0,Recursos de pessoas jurídicas,Nao especificado,...,ELEITO POR QP,2017,43283811001806,ADILSON ROSSI,KALUNGA COM INDUSTRIA GRAFICA LTDA,300432,7,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,286.50,43283811
316,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,2635.0,Recursos de pessoas jurídicas,Nao especificado,...,ELEITO POR QP,2017,43283811001806,ADILSON ROSSI,KALUNGA COM INDUSTRIA GRAFICA LTDA,300432,9,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,108.60,43283811
317,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,2635.0,Recursos de pessoas jurídicas,Nao especificado,...,ELEITO POR QP,2017,43283811001806,ADILSON ROSSI,KALUNGA COM INDUSTRIA GRAFICA LTDA,300432,10,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,676.00,43283811
318,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,2635.0,Recursos de pessoas jurídicas,Nao especificado,...,ELEITO POR QP,2017,43283811001393,ADILSON ROSSI,KALUNGA COM.E IND.GRÁFICA LTDA,300432,12,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,229.20,43283811
319,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,2635.0,Recursos de pessoas jurídicas,Nao especificado,...,ELEITO POR QP,2017,43283811002284,ADILSON ROSSI,KALUNGA COM E IND. GRAFICA LTDA,300432,12,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,669.16,43283811
320,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,2635.0,Recursos de pessoas jurídicas,Nao especificado,...,ELEITO POR QP,2017,43283811006190,ADILSON ROSSI,KALUNGA COM E IND GRAFICA LTDA,300432,12,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,459.29,43283811
321,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,2635.0,Recursos de pessoas jurídicas,Nao especificado,...,ELEITO POR QP,2018,43283811001806,ADILSON ROSSI,KALUNGA COM INDUSTRIA GRAFICA LTDA,300432,2,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,108.50,43283811
322,SP,PSDB,Deputado Estadual,FERNANDO CAPEZ,08238377854,43283811000150,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA,2635.0,Recursos de pessoas jurídicas,Nao especificado,...,ELEITO POR QP,2018,43283811001806,ADILSON ROSSI,KALUNGA COM INDUSTRIA GRAFICA LTDA,300432,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,108.40,43283811


In [ ]:
# Soma de despesas pelo nome do fornecedor/doador

In [59]:
despesas_doadores_diretos_f.groupby("Fornecedor").valor_fornecedor.sum().reset_index().sort_values("valor_fornecedor", ascending=False)

,Fornecedor,valor_fornecedor
60,EXPRESSÃO SANTO ANDRE GRAFICA E EDITORA LTDA EPP,144000.00
2,ALPHAMARK SOLUÇÕES LTDA,140895.00
68,GRÁFICA TRES PONTAS LTDA - EPP,134810.00
57,CRONY ARTES GRAFICAS LTDA ME,125978.40
70,HANNA INCORPORAÇÕES E VENDAS LTDA,112150.00
63,FERNANDO LUIZ LAUDINO & CIA LTDA - ME,88630.58
1,ALPHAMARK PROPAGANDA E MARKETING LTDA,83520.00
66,GRAFICA TRES PONTAS LTDA - EPP,80626.40
59,EXPRESSAO SANTO ANDRE GRAFICA EDITORA LTDA EPP,71196.00
65,GRAFICA ELITE,67080.00


In [62]:
despesas_doadores_diretos_f.to_csv('resultados/despesas_deputados_sp_fornecedores_doadores.csv', index=False, decimal = ',')

# PF

In [63]:
despesas_doadores_diretos_f_PF = pd.merge(eleitos_d_doadores.drop_duplicates('cpf_cnpj_doador'), despesas_atuaisPF, left_on='cpf_cnpj_doador', right_on='CNPJ')

In [64]:
# Soma de despesas dos doadores que são PF

In [65]:
despesas_doadores_diretos_f_PF['valor_fornecedor'].sum()

581425.3099999999

In [66]:
despesas_doadores_diretos_f_PF.groupby("Fornecedor").valor_fornecedor.sum().reset_index().sort_values("valor_fornecedor", ascending=False)

,Fornecedor,valor_fornecedor
7,ROBERTO TAVARES DE CARVALHO,174001.58
4,EUVALDO RODRIGUEZ ALBALADEJO,113786.33
2,CLEUSA ANANIAS DE ARAUJO,89900.00
0,ANTONIO CABEZA SASTRE,65540.00
3,CLOVIS SILLES GALVÃO,65540.00
5,MARIA ELISA INTERNODATO NUNES VIANA,52664.50
6,MARIA ELISA INTERNODATO NUNES VIEIRA,17492.90
1,CIRO BERBES JUNIOR,2500.00


In [67]:
nome = despesas_doadores_diretos_f_PF[(despesas_doadores_diretos_f_PF['Fornecedor'] == 'ROBERTO TAVARES DE CARVALHO')]
nome

,uf_x,partido_eleicao_x,cargo_x,nome_completo_x,cpf,cpf_cnpj_doador,nome_doador,valor_doado,tipo_receita,fonte_recurso,...,situacao,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,valor_fornecedor,cnpj_raiz_fornecedor
141,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,2000.0,Recursos de pessoas físicas,Nao especificado,...,ELEITO POR QP,2015,16381726849,GILENO GOMES,ROBERTO TAVARES DE CARVALHO,300546,5,L - LOCAÇÃO DE BENS IMÓVEIS,2400.00,documento_cpf_
142,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,2000.0,Recursos de pessoas físicas,Nao especificado,...,ELEITO POR QP,2015,16381726849,GILENO GOMES,ROBERTO TAVARES DE CARVALHO,300546,6,L - LOCAÇÃO DE BENS IMÓVEIS,2400.00,documento_cpf_
143,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,2000.0,Recursos de pessoas físicas,Nao especificado,...,ELEITO POR QP,2015,16381726849,GILENO GOMES,ROBERTO TAVARES DE CARVALHO,300546,7,L - LOCAÇÃO DE BENS IMÓVEIS,2400.00,documento_cpf_
144,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,2000.0,Recursos de pessoas físicas,Nao especificado,...,ELEITO POR QP,2015,16381726849,GILENO GOMES,ROBERTO TAVARES DE CARVALHO,300546,8,L - LOCAÇÃO DE BENS IMÓVEIS,2400.00,documento_cpf_
145,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,2000.0,Recursos de pessoas físicas,Nao especificado,...,ELEITO POR QP,2015,16381726849,GILENO GOMES,ROBERTO TAVARES DE CARVALHO,300546,9,L - LOCAÇÃO DE BENS IMÓVEIS,2400.00,documento_cpf_
146,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,2000.0,Recursos de pessoas físicas,Nao especificado,...,ELEITO POR QP,2015,16381726849,GILENO GOMES,ROBERTO TAVARES DE CARVALHO,300546,10,L - LOCAÇÃO DE BENS IMÓVEIS,2400.00,documento_cpf_
147,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,2000.0,Recursos de pessoas físicas,Nao especificado,...,ELEITO POR QP,2015,16381726849,GILENO GOMES,ROBERTO TAVARES DE CARVALHO,300546,11,L - LOCAÇÃO DE BENS IMÓVEIS,2400.00,documento_cpf_
148,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,2000.0,Recursos de pessoas físicas,Nao especificado,...,ELEITO POR QP,2015,16381726849,GILENO GOMES,ROBERTO TAVARES DE CARVALHO,300546,12,L - LOCAÇÃO DE BENS IMÓVEIS,2400.00,documento_cpf_
149,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,2000.0,Recursos de pessoas físicas,Nao especificado,...,ELEITO POR QP,2016,16381726849,GILENO GOMES,ROBERTO TAVARES DE CARVALHO,300546,1,L - LOCAÇÃO DE BENS IMÓVEIS,2400.00,documento_cpf_
150,SP,PRB,Deputado Estadual,JORGE WILSON GONÇALVES DE MATTOS,05066887806,16381726849,ROBERTO TAVARES DE CARVALHO,2000.0,Recursos de pessoas físicas,Nao especificado,...,ELEITO POR QP,2016,16381726849,GILENO GOMES,ROBERTO TAVARES DE CARVALHO,300546,2,L - LOCAÇÃO DE BENS IMÓVEIS,4800.00,documento_cpf_


In [68]:
despesas_doadores_diretos_f_PF.to_csv('resultados/despesas_deputados_sp_fornecedoresPF_doadores.csv', index=False, decimal = ',')